In [1]:
import httpx

In [ ]:
import json

In [ ]:
# with open("/Users/steffen/sim_company/chemical/raw/assets.json", "r") as f:
#     data = json.load(f)

In [ ]:
# mapping = [
#     ("temp vor Rückstandskühler", "TI-T0022"),
#     ("temp nach Rückstandskühler", "TI-T0026"),
#     ("stand nach Rückstandskühler", "LI-L0009"),
#     ("temp nach Destillatkühler a", "TI-T0027"),
#     ("temp nach Destillatkühler b", "TI-T0028"),
#     ("stand nach Destillatkühler a", "LI-L0010"),
#     ("stand nach Destillatkühler b", "LI-L0008"),
#     ("druck nach Destillatkühler a", "PI-P0020"),
#     ("druck nach Destillatkühler b", "PI-P0019"),
#     ("menge an zufluss in tanklager", "FQI-F0024"),
#     ("dampfdruck in der destillation", "PDI-P0018"),
#     ("obere Temperatur in der destillation", "TRC-T0029"),
#     ("mittlere Temperatur in der destillation", "TRC-T0031"),
#     ("untere Temperatur in der destillation", "TRC-T0032"),
#     ("menge in der destillation", "LRC-L0011"),
#     ("zuflussmenge zur destillation", "FRC-F0025"),
#     ("zuflusstemperatur zur destillation", "TI-T0030"),
#     ("temp Kühlwasser-Rücklauf von der Destillation", "TI-T0023"),
#     ("menge Kühlwasser-Rücklauf von der Destillation", "FI-F0023"),
#     ("temp Kühlwasser-Rücklauf von Kondensator", "TI-T0023"),
#     ("menge Kühlwasser-Rücklauf von Kondensator", "FI-F0023"),
#     ("temp Kühlgas-Rücklauf von der Destillation", "TI-T0025"),
#     ("temp Kühlgas-Rücklauf von Gaskühler", "TI-T0025"),
# ]

In [ ]:
# raw = []
# for desc, tag in mapping:
#     response = httpx.get(f"http://localhost:5050/v1/id_from_name/{tag}")
#     _id = response.json()
#     raw.append({"description": desc, "name": tag, "tag": tag, "id": _id})

In [ ]:
# with open("/Users/steffen/sim_company/chemical/raw/knowledge_base.json", "w") as f:
#     json.dump(raw,f)

In [ ]:
test = httpx.get("http://localhost:5051/v1/embedding/test")

In [ ]:
n_emb = len(test.json()["embedding"])

## Data ingestion

In [ ]:
with open("/Users/steffen/sim_company/chemical/raw/knowledge_base.json", "r") as f:
    data = json.load(f)

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(host="localhost", port=6333)


In [ ]:
from qdrant_client.models import VectorParams, Distance

# client.delete_collection(collection_name="chemical")


if not client.collection_exists("chemical"):
   client.create_collection(
      collection_name="chemical",
      vectors_config=VectorParams(size=n_emb, distance=Distance.COSINE),
   )


In [ ]:
# import numpy as np
# from qdrant_client.models import PointStruct

# for idx, _data in enumerate(data):

#     response = httpx.get(f"http://localhost:5051/v1/embedding/{_data["description"]}")
#     embedding = response.json()["embedding"]
    
#     client.upsert(
#         collection_name="chemical",
#         points=[
#             models.PointStruct(
#                 id=_data["id"],
#                 payload=_data,
#                 vector=embedding,
#             ),
#         ],
#     )



## Data retrieval

In [2]:
from pydantic import BaseModel

In [3]:
class KBResponse(BaseModel):
    query: str
    text: str
    score: float
    id: str
    tag: str
    name: str

In [4]:
query_text = "What is the temperature?"

response = httpx.get(f"http://localhost:5051/v1/embedding/", params={"text": query_text})
embedding = response.json()["embedding"]

In [5]:

response = httpx.post(f"http://localhost:5051/v1/search/", json={"embedding": embedding})
response = response.json()

In [6]:
scores = []

for p in response["results"]:
    r = httpx.get(f"http://localhost:5051/v1/ranking/", params={"text": p["description"], "query": query_text})
    score = r.json()
    p.pop("score", None)
    data = KBResponse(**score, **p)
    scores.append(data)


In [7]:
scores

[KBResponse(query='What is the temperature?', text='temperature in the upper part of the distillation column', score=-5.1993913650512695, id='54fc69f2-bc6c-4bbb-a22f-c07c975a0386', tag='TRC-T0029', name='TRC-T0029'),
 KBResponse(query='What is the temperature?', text='temperature in the lower part of the distillation column', score=-5.628674507141113, id='59c3f5c6-6fe1-48ed-85c8-b7e99b5a3e02', tag='TRC-T0032', name='TRC-T0032'),
 KBResponse(query='What is the temperature?', text='temperature in the middle part of the distillation column', score=-5.320639133453369, id='22a9f4d8-b297-4d17-9ab0-8461e0683018', tag='TRC-T0031', name='TRC-T0031'),
 KBResponse(query='What is the temperature?', text='measures the cooling water temperature before the residue cooler', score=-8.388864517211914, id='cec684e7-cf21-47fe-8213-031cd576163c', tag='TI-T0022', name='TI-T0022'),
 KBResponse(query='What is the temperature?', text='temperature of the cooling water flow away from the condenser', score=-6.802